In [1]:
import pandas as pd
from datetime import datetime
import requests
import time 
from requests_html import HTMLSession
from PIL import Image
from io import BytesIO

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import display, clear_output

In [ ]:
!pip install selenium

In [ ]:
! pip install --upgrade selenium

In [2]:
s = Service('chromedriver')
driver = webdriver.Chrome(service=s)

In [ ]:
driver.get("https://bikroy.com/bn/ads/bangladesh/mobiles")

In [ ]:
driver.close()

In [ ]:
driver.title

In [ ]:
driver.back()

In [ ]:
driver.forward()

In [ ]:
product_link = driver.find_elements(by=By.CLASS_NAME, value="card-link--3ssYv")
product_link

In [ ]:
product_urls = [p.get_attribute('href') for p in product_link]
product_urls

In [ ]:
for p in product_urls[:5]:
    print(p)

In [ ]:
driver.get('https://bikroy.com/bn/ad/realme-x7-5g-6128-used-for-sale-barishal')

In [ ]:
title_ele = driver.find_element(by=By.TAG_NAME, value='h1')
title_ele.text

In [ ]:
descript_ele = driver.find_element(by=By.XPATH, value='//*[@id="collapsible-content-0"]/ul/div')
descript_ele

In [ ]:
descript_ele.text.split('\n')

In [ ]:
'বক্স নাই, ব্যাক কাভার আছে। চার্জার আছে। পলি লাগানো আছে। Amoled Display Indisplay Fingerprint Dimensity 800U octa core processor '

In [ ]:
' '.join(descript_ele.text.split('\n'))

In [ ]:
import time

In [ ]:
s = Service('chromedriver')
driver = webdriver.Chrome(service=s)
# session = HTMLSession()
start = time.time()
pages = 5
single_page_urls = []
page_cnt, url_cnt = 0, 0
for p in range(1, pages+1):
    bikroy_url = f'https://bikroy.com/en/ads/bangladesh/home-living?sort=date&order=desc&buy_now=0&urgent=0&page={p}'
    print(bikroy_url)


#     r = session.get('http://localhost:8050/render.html', params={'url': bikroy_url, 'wait': 1})
#     print(r.status_code)

    driver.get(bikroy_url)
    time.sleep(3)
    page_cnt += 1
    for i in range (1, 28):
        try:
            single_url = driver.find_element(by=By.XPATH, value=f'//*[@id="app-wrapper"]/div[1]/div[3]/div/div[2]/div[4]/div[2]/div/div[1]/div[1]/ul/li[{i}]/a').get_attribute('href')
#             single_url = r.html.find(f'#app-wrapper > div.container--297Nx.all > div:nth-child(3) > div > div.row--3Vhjr.main-container--28aG5.justify-content-flex-start--1Xozy.align-items-normal--vaTgD.flex-wrap-wrap--2PCx8.flex-direction-row--27fh1.flex--3fKk1 > div.sm-col-12--30zDS.lg-col-9--20qCf.block--3v-Ow > div.ad-list-container--1UnyA > div > div.ad-list--2Y3ql > div.list-wrapper--t_A02 > ul > li:nth-child({i}) > a')
#             single_url = single_url[0].attrs['href']
            single_page_urls.append(single_url)
            url_cnt += 1
            print("Page: " + str(page_cnt) + " URL: " + str(url_cnt))
        except Exception as e:
            print(e) 
            print('Not details link')
    
#     clear_output(wait=True)
print(time.time()-start)
# driver.maximize_window()

In [ ]:
len(single_page_urls), single_page_urls

In [ ]:
url = 'https://bikroy.com/en/ad/microtek-solar-ips-1435-850w-12v-10-led-5-fan-for-sale-dhaka'
driver.get(url)
x = show_more_click(driver)
x

In [ ]:
temp = single_url[0].attrs['href']
temp

In [ ]:
with open('category_url/home-living_product_urls.txt', 'w') as f:
    for s in single_page_urls:
        f.write(s)
        f.write('\n')

In [3]:
def title_scrap(driver):
    try:
        title = driver.find_element(by=By.TAG_NAME, value='h1').text
        print("Title: ", title)
        return title        
    except:
        print("Product Not found. May be sold out!")
        return None

In [4]:
def time_location_scrap(driver):
    time_location = driver.find_element(by=By.CLASS_NAME, value='sub-title--37mkY').text
    print("Time & Location: ", time_location)

    return time_location

In [5]:
def price_scrap(driver):
    price = driver.find_element(by=By.CLASS_NAME, value='amount--3NTpl').text
    print("Price: ", price)

    return price

In [14]:
def image_scrap(driver, category):
    try:
        images = driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[1]/div/div/ul')
        img_path = []
        for img in images.find_elements(by=By.TAG_NAME,value='img'):
            img_urls = img.get_attribute('src') ## main link = img_urls

            img_urls_firstname = '/'.join(img_urls.split('/')[:-3])## main link thak 3 ta character bad
            

            
            img_urls_rename = img_urls_firstname + '/620/466/fitted.jpg'## main link e 3 ta character plus
                                                                        ## ei khane ekta new link create korlo
           

            img_name = '@'.join(img_urls_firstname.split('/')[-2:])## -2 index thaka string ta start hoise. and itz img_name

            r = requests.get(img_urls_rename)
            i = Image.open(BytesIO(r.content))
            i.save(f"E:/DS - IIUC/Python/classes/second class scraping/image/{category}/{img_name}.jpg")
            img_path.append(img_name)
        return img_path
    except:
        print('No images')        
        return 'N/A'

In [7]:
def meta_data_scrap(driver):
    try:
        meta_data = driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[2]/div[2]')

        md = meta_data.find_elements(by=By.TAG_NAME,value='div')
        meta_dic = {}
        for m in md:
            m = m.text
            if '\n' in m:
                key, val = m.split('\n')[0].replace(':', ''), m.split('\n')[1]
                meta_dic[key] = val
        print("Meta Data: ", meta_dic)
        return meta_dic
    except:
        print('No meta data found')        
        return 'N/A'

In [8]:
def show_more_click(driver):    
    try:
        s_driver.find_element(by=By.CLASS_NAME, value='show-text--13FcL').click()
#         s_driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[2]/div[3]/div/div[2]/div/div[2]/button').click()
        time.sleep(2)
    except:
        print("No 'Show more' button")

In [13]:
def feature_scrap(driver):
    try:
        feature = driver.find_element(by=By.CLASS_NAME, value='text--Xap4O').text
#         feature = s_driver.find_element(by=By.XPATH, value='//*[@id="app-wrapper"]/div[1]/div[2]/div[2]/div[1]/div[1]/div/div[1]/div/div[2]/div[4]/p').text
#         feature = feature.find_elements(by=By.TAG_NAME,value='p').text
        print('Features: ', feature)
        return feature
    except:
        print("No feature")        
        return 'N/A'

In [9]:
def description_scrap(driver):
    # desc = s_driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div[2]/div[2]/div[3]/div[2]/div/div[1]/div/div[2]/div[3]/div/div[2]/div/div[1]/div/ul/div')
    # description[0].text
    # description = [d.text.replace("'\uf076", "").replace("''\uf0d8", "") for d in desc.find_elements_by_tag_name('p')]
    # print(description)

    try:
        desc = driver.find_element(by=By.CLASS_NAME, value='description--1nRbz').text
        print("Description: ", desc)
        return desc
    except:
        print("No description")        
        return 'N/A'

In [10]:
def open_driver():
    op = webdriver.ChromeOptions()
    op.add_argument('headless')
    s = Service('chromedriver')
    driver = webdriver.Chrome(service=s, options=op)
    return driver

In [11]:
def product_scrap(driver, url, category):
    driver.get(url)
    time.sleep(5)

    # single page data scrapping
    title = title_scrap(driver)
    # if title not found (product sold) don't proceed further for other data
    if not title:
        return [None]*7
    # scrapping time and location
    time_location = time_location_scrap(driver)
    
    # scrapping price
    price = price_scrap(driver)
    
    # scrapping all images
    images = image_scrap(driver, category)
    

    # extractig meta data
    meta_data = meta_data_scrap(driver)
        
    # click on show_more button
    show_more_click(driver)

    # extracting feature data
    feature = feature_scrap(driver)
    
    # extracting description
    description = description_scrap(driver)
    
    return title, time_location, price, images, meta_data, feature, description

In [15]:
category = 'mobile'
with open('mobile_product_urls.txt', 'r') as f:
    single_url = f.readlines()
    titles, locations, prices, img_lst, meta_lst,  feature_lst, desc_lst = [], [], [], [], [], [], []
    cnt = 1
#     print(single_url)
    for s in single_url:
        print(cnt, s)
        driver = open_driver()
        
        title, time_location, price, images, meta_data, feature, description = product_scrap(driver, s, category)
        if title:
            titles.append(title)
            locations.append(time_location)
            prices.append(price)
            img_lst.append(images)
            meta_lst.append(meta_data)
            feature_lst.append(feature)
            desc_lst.append(description) 

            cnt += 1
            print("=============================================")
            clear_output(wait=True)

3 https://bikroy.com/en/ad/computer-table-for-sale-sylhet-566
Title:  Computer Table
Time & Location:  Posted on 11 Oct 1:41 am, Balaganj, Sylhet
Price:  Tk 1,550
Meta Data:  {'Condition': 'Used', 'Furniture type': 'Tables & Chairs'}
No 'Show more' button
No feature
Description:  1 Year used.


In [ ]:
driver.get('https://bikroy.com/bn/ad/realme-x7-5g-6128-used-for-sale-barishal')
desc = driver.find_element(by=By.CLASS_NAME, value='description--1nRbz').text
desc


In [ ]:
df_lst = [titles, locations, prices, img_lst, meta_lst,  feature_lst, desc_lst, single_url[:2]]
column_lst = ['Title', 'Location', 'Price', 'Images', 'Meta Data', 'Feature', 'Description', 'URL']
df = pd.DataFrame (df_lst).T
df.columns = column_lst
df.to_csv(f'csv/{category}.csv', index=False)
df.head()


In [ ]:
titles